# Code below are the "main" functions, what I run to execute data collection

In [1]:
import Maxpreps.py
import On3_website.py
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
from requests.exceptions import HTTPError
import numpy as np
import re
import time
import sys
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

ModuleNotFoundError: No module named 'Maxpreps'

In [19]:
dfs = []
positions = ['qb', 'rb', 'wr', 'te', 'ot', 'iol', 'edge', 'dl', 'lb', 'cb', 's']

In [20]:
off_positions = ['QB', 'RB', 'WR', 'TE', 'T', 'C', 'G']
def_positions = ['DE', 'CB', 'FS', 'SS', 'DT', 'MLB', 'OLB']
maxpreps_positions = np.concatenate((off_positions, def_positions))
years = [2023, 2024, 2025, 2026]

for year in years:
    for pos in positions:
        position_dfs = {}
        dfs = []
        for i in range(1, 6):
            url = f"https://www.on3.com/db/rankings/industry-comparison/football/{year}/?position={pos}&page={i}"
            dfs.append(get_player_ratings_and_school(url))

        position_dfs[pos] = pd.concat(dfs)
        position_dfs[pos].reset_index(drop=True, inplace=True)
        names = list(position_dfs[pos]["Player_Name"])
        #set up dataframe

        for i, name in enumerate(names):
            print(name, i) #shows the player collecting data for
            try:
                time.sleep(2.5 + 0.02 * i)
                soup = get_first_google_result(name + " " + position_dfs[pos].at[i, 'On3SchN'] + " maxpreps")
                school_name = soup.find('div', class_ = 'school-name')
                school_loc = soup.find('div', class_ = "location")
                if school_name and school_loc:
                    position_dfs[pos].at[i, "MPSchN"] = school_name.text + school_loc.text
                #assign school name
                
                jersey_pos = find_pos_on_maxpreps(name, soup, pos) 
                position_dfs[pos].at[i, "On3_position"] = pos
                fill_df_position_inplace(position_dfs[pos])
                #assign player position
                try:
                    position_dfs[pos].at[i, "Team_Captain"] = get_team_captain(soup)
                    position_dfs[pos].at[i, "MPTmRtg"] = get_team_rating(soup)
                    position_dfs[pos].at[i, "MP_potg_count"], position_dfs[pos].at[i, "MP_potg_count_total"] = get_mp_potg(soup)
                    position_dfs[pos].at[i, "conference_name"] = get_mp_confName(soup)
                    #assign more player info
                except TypeError:
                    fill_df_error_inplace(position_dfs, pos, 'TypeError')

            except HTTPError:
                print('Error encountered')
                fill_df_error_inplace(position_dfs, pos, 'HTTPError')

            if i % 40 == 0:
                time.sleep(10 + 0.05 * i)

        position_dfs[pos].to_csv(f"new_{year}_" + pos + "_ratings.csv", index=False)

Arch Manning 0
[]
Couldn't find team captain for player above
Dante Moore 1
[<div class="sport"><div><a class="sc-a2e14072-0 hCsilC" href="https://www.maxpreps.com/mi/detroit/king-crusaders/football/22-23/schedule/" style="--data-color:#004ACE;--data-hover-color:#002cb0">King Varsity Football</a></div><div>#5 • QB</div></div>]
Nico Iamaleava 2
[<div class="sport"><div><a class="sc-a2e14072-0 hCsilC" href="https://www.maxpreps.com/ca/downey/warren-bears/football/21-22/schedule/" style="--data-color:#004ACE;--data-hover-color:#002cb0">Warren Varsity Football</a></div><div>#8 • QB</div></div>]
Jackson Arnold 3
[<div class="sport"><div><a class="sc-a2e14072-0 hCsilC" href="https://www.maxpreps.com/tx/denton/guyer-wildcats/football/22-23/schedule/" style="--data-color:#004ACE;--data-hover-color:#002cb0">Guyer Varsity Football</a></div><div>#11 • QB</div></div>]
Malachi Nelson 4
[<div class="sport"><div><a class="sc-a2e14072-0 hCsilC" href="https://www.maxpreps.com/ca/los-alamitos/los-alamit

KeyboardInterrupt: 